In [1]:
import random
import numpy as np
import pandas as pd
import cv2
import os
import ast
import shutil
from IPython.display import display
import matplotlib.pyplot as plt

In [2]:
shutil.copyfile("../input/coral-yaml/Arial.ttf", "./Arial.ttf")

'./Arial.ttf'

In [3]:
import torch
ckpt_path = "../input/coral-yaml/best_yolov5m6_0.95_precision_fanciful_monkey_39.pt"

model = torch.hub.load('../input/yolov5', 'custom',
                           path=ckpt_path,
                           source='local',
                       force_reload=True)

YOLOv5 🚀 2022-2-19 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 378 layers, 35248920 parameters, 0 gradients
Adding AutoShape... 


In [4]:
def result_string_calculator(results_dataframe):
    if results_dataframe.shape[0] != 0:
        total_output = str()
        for index,row in results_dataframe.iterrows():
            x_center = row['xmin'] + ((row['xmax'] - row['xmin'])/2)
            y_center = row['ymin'] + ((row['ymax'] - row['ymin'])/2)
            height= row['ymax'] - row['ymin']
            width = row['xmax'] - row['xmin']
            confidence_score = round(row['confidence'], 3)
            output_string = "{} {} {} {} {} ".format(confidence_score,round(x_center),round(y_center),round(width),round(height))
            total_output += output_string
    else:
        total_output = "0 0 0 0 0"
    return total_output

In [5]:
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [6]:
for (pixel_array, sample_prediction_df) in iter_test:
    results = model(pixel_array)
    results_df=results.pandas().xyxy[0]
    sample_prediction_df['annotations'] =  result_string_calculator(results_df, height, width) # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions